## KOUSHIK KANEPALLY - 017
## CH. VAMSHI - 010
## V NARAYANAN - 019

In [15]:
import numpy as np
import random
import json
import nltk
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
nltk.download('punkt')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\narayanan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True


### Creating Custom Functions:
We will create custom Functions so that it is easy for us to implement afterwards.

In [16]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)
#tokenizeit will divide sentence to words

def stem(word):
    return stemmer.stem(word.lower())


In [17]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


In [18]:
##BAG OF WORDS

In [19]:

def bag_of_words(tokenized_sentence, words):
    
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1
    

    return bag

In [20]:
with open('Intent.json', 'r') as f:
    intents = json.load(f)
intents


{'intents': [{'intent': 'Greeting',
   'text': ['Hi',
    'Hi there',
    'Hola',
    'Hello',
    'Hello there',
    'Hya',
    'Hya there'],
   'responses': ['Hi human, please tell me your GeniSys user',
    'Hello human, please tell me your GeniSys user',
    'Hola human, please tell me your GeniSys user']},
  {'intent': 'GreetingResponse',
   'text': ['My user is Adam',
    'This is Adam',
    'I am Adam',
    'It is Adam',
    'My user is Bella',
    'This is Bella',
    'I am Bella',
    'It is Bella'],
   'responses': ['Great! Hi <HUMAN>! How can I help?',
    'Good! Hi <HUMAN>, how can I help you?',
    'Cool! Hello <HUMAN>, what can I do for you?',
    'OK! Hola <HUMAN>, how can I help you?',
    'OK! hi <HUMAN>, what can I do for you?']},
  {'intent': 'CourtesyGreeting',
   'text': ['How are you?',
    'Hi how are you?',
    'Hello how are you?',
    'Hola how are you?',
    'How are you doing?',
    'Hope you are doing well?',
    'Hello hope you are doing well?'],
   'respo


In order to get the right information, we will be unpacking it with the following code. 


In [21]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['intent']
    # add to tag list
    tags.append(tag)
    for pattern in intent['text']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))



This will separate all the tags & words into their separate lists



### Cleaning & Preparing the data using our custom functions

We will be cleaning the data by implementing the functions we created in our previous cells.

In [22]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
#26 rows
print(len(tags), "tags:", tags)
#7 classes ->7  output neurons in the final layer
print(len(all_words), "unique stemmed words:", all_words)
# 54 input neurons in the input layer
xy

143 patterns
22 tags: ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye', 'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'WhoAmI']
114 unique stemmed words: ["'s", ',', 'a', 'about', 'adam', 'adio', 'am', 'ani', 'anyon', 'are', 'awar', 'bay', 'be', 'bella', 'bore', 'by', 'bye', 'call', 'camera', 'can', 'cheer', 'clever', 'commun', 'comprendo', 'consciou', 'could', 'do', 'door', 'enough', 'for', 'friend', 'fuck', 'geniou', 'get', 'girl', 'give', 'good', 'goodby', 'gossip', 'got', 'great', 'have', 'hear', 'hello', 'help', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'identifi', 'in', 'intellig', 'is', 'it', 'joke', 'know', 'later', 'laugh', 'make', 'me', 'mean', 'meant', 'more', 'my', "n't", 'name', 'need', 'not', 'off', 'ok', 'open', 'pleas', 'pod', 'prove', 'quiet', 'real'

[(['Hi'], 'Greeting'),
 (['Hi', 'there'], 'Greeting'),
 (['Hola'], 'Greeting'),
 (['Hello'], 'Greeting'),
 (['Hello', 'there'], 'Greeting'),
 (['Hya'], 'Greeting'),
 (['Hya', 'there'], 'Greeting'),
 (['My', 'user', 'is', 'Adam'], 'GreetingResponse'),
 (['This', 'is', 'Adam'], 'GreetingResponse'),
 (['I', 'am', 'Adam'], 'GreetingResponse'),
 (['It', 'is', 'Adam'], 'GreetingResponse'),
 (['My', 'user', 'is', 'Bella'], 'GreetingResponse'),
 (['This', 'is', 'Bella'], 'GreetingResponse'),
 (['I', 'am', 'Bella'], 'GreetingResponse'),
 (['It', 'is', 'Bella'], 'GreetingResponse'),
 (['How', 'are', 'you', '?'], 'CourtesyGreeting'),
 (['Hi', 'how', 'are', 'you', '?'], 'CourtesyGreeting'),
 (['Hello', 'how', 'are', 'you', '?'], 'CourtesyGreeting'),
 (['Hola', 'how', 'are', 'you', '?'], 'CourtesyGreeting'),
 (['How', 'are', 'you', 'doing', '?'], 'CourtesyGreeting'),
 (['Hope', 'you', 'are', 'doing', 'well', '?'], 'CourtesyGreeting'),
 (['Hello', 'hope', 'you', 'are', 'doing', 'well', '?'], 'Courte


### Creating Training Data: 
We will transform the data into a format that our PyTorch Model can Easily Understand


In [23]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
y_train


array([ 7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  2,  2,
        2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4, 10, 10, 10, 10, 10, 10, 14, 14, 14, 14, 14, 14, 14, 19,
       19, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 11, 11, 11, 11, 11,
       11, 11, 20, 20, 20, 20, 20, 20, 16, 16, 16, 16, 16, 16, 16, 17, 17,
       17, 17,  5,  5,  5,  5,  1,  1,  1,  1,  1,  1, 21, 21, 21, 21, 21,
       21,  0,  0,  0,  0,  0,  0,  0,  6,  6,  6,  6,  6,  6,  9,  9,  9,
        9,  9,  9, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13,
       15, 15, 15, 15, 15, 15, 15])


    `One hot encoding Is the process of splitting multiclass or multi valued data column to separate columns and labelling the cell 1 in the row where it exists. (we won’t use it so don’t worry about it)`


In [24]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        #input layer to hidden layer
        
        #hidden layer to hidden layer
        self.l2 = nn.Linear(hidden_size, num_classes)
        #hidden layer to output layer
        self.relu = nn.ReLU()
    
    def forward(self, x):
        #send to 1st hidden layer
        out = self.l1(x)
        #activate 1st hidden
        out = self.relu(out)
        
          #send to output layer
        out = self.l2(out)

        return out

In [25]:
class ChatDataset(Dataset):

    def __init__(self):
        #26 samples
        self.n_samples = len(X_train)
        #26 samples with 1 hort encoding
        self.x_data = X_train
        #26 smapoles with class number
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [26]:
#26 rows
#8*3+2 =  1 epoch
#weight is getting updated after 8 rows
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

114 22


In [27]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for s,y in train_loader:
  print(len(s[0]))
  print(y)

114
tensor([ 4,  6, 18,  3,  2,  9, 14, 14], dtype=torch.int32)
114
tensor([19,  5, 13, 20,  9,  7, 19, 12], dtype=torch.int32)
114
tensor([15, 18,  6, 18,  3, 11, 10, 18], dtype=torch.int32)
114
tensor([21, 16, 10, 11,  9, 11, 21, 14], dtype=torch.int32)
114
tensor([ 7,  8, 10,  3,  5,  1, 16, 11], dtype=torch.int32)
114
tensor([20,  0,  0, 15,  9, 15, 16,  2], dtype=torch.int32)
114
tensor([11,  6, 13,  4, 13, 12, 12, 20], dtype=torch.int32)
114
tensor([ 2, 19, 13,  1, 14,  1,  3, 16], dtype=torch.int32)
114
tensor([ 9, 17,  6, 12,  5, 10, 15, 19], dtype=torch.int32)
114
tensor([ 3, 13,  0, 17,  7,  0, 10, 10], dtype=torch.int32)
114
tensor([ 4, 11, 11, 15,  7, 20,  3, 17], dtype=torch.int32)
114
tensor([ 8, 21,  9, 18, 16,  8,  6, 12], dtype=torch.int32)
114
tensor([ 1,  6,  8, 21,  3, 14, 21, 19], dtype=torch.int32)
114
tensor([ 8, 15,  8,  4, 20,  4,  8,  5], dtype=torch.int32)
114
tensor([ 0,  0, 17, 21,  2,  2, 15,  2], dtype=torch.int32)
114
tensor([13, 16,  2,  1, 14, 14,  1, 


## Training the Model
---


In [28]:

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

Epoch [100/1000], Loss: 0.2901
Epoch [200/1000], Loss: 0.0407
Epoch [300/1000], Loss: 0.0130
Epoch [400/1000], Loss: 0.0034
Epoch [500/1000], Loss: 0.0012
Epoch [600/1000], Loss: 0.0007
Epoch [700/1000], Loss: 0.0001
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
final loss: 0.0000


In [29]:

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


training complete. file saved to data.pth



## Loading our Saved Model
---

In [30]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('Intent.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


NeuralNet(
  (l1): Linear(in_features=114, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=22, bias=True)
  (relu): ReLU()
)


 
## Using the Chatbot:


In [ ]:

bot_name = "Amrita Chatbot"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    print(X)
    X = X.reshape(1, X.shape[0])
    print(X)
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)
    print(predicted)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    print(probs)
    prob = probs[0][predicted.item()]
    print(prob.item())
    if prob.item() > 0.75:
        for intent in intents['intents']:
            
            if tag == intent["intent"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: what time is it?
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
tensor([19], device='cuda:0')
tensor([[2.2641e-12, 1.5600e-16, 1.3842e-10, 4.0083e-07, 8.9257e-11, 4.0634e-22,
         3.4643e-10, 4.1802e-24, 1.5167e-08, 1.0921e-08, 6.2749e-08, 6.1850e-17,
         6.2653e-12, 1.8912e-10, 2.0086e-15, 2.309